# 0. Importing Necessary Packages

In [1]:
# Printing the information of Python, IPython, OS, and the generation date.
# Before running this, you have to install version_information module with "pip install version_information".
%load_ext version_information
%version_information

Software versions
Python 2.7.17 64bit [GCC 7.3.0]
IPython 5.8.0
OS Linux 5.8.18 100.fc31.x86_64 x86_64 with fedora 31 Thirty_One
Mon Apr 25 07:55:31 2022 KST

In [2]:
# Printing the versions of packages
from importlib_metadata import version
for pkg in ['numpy', 'matplotlib', 'pandas', 'astropy', 'pyraf']:
    print(pkg+": ver "+version(pkg))

numpy: ver 1.16.5
matplotlib: ver 2.2.3
pandas: ver 0.24.2
astropy: ver 2.0.16
pyraf: ver 2.1.15


In [3]:
# Matplotlib backend
%matplotlib notebook

# Importing necessary modules
import time
import numpy as np
import glob, os
from matplotlib import pyplot as plt
import pandas as pd
from astropy.io import fits
from astropy.stats import sigma_clipped_stats

current_dir = os.getcwd()    # Current working directory
login_file = "/data/jlee/DATA/TA/2022A/login.cl"    # Please give the absolute path of your 'login.cl' file

# Reading 'login.cl' file
f = open(login_file, "r")
ll = f.readlines()
f.close()

# Finding the string for home directory in 'login.cl' file
line_homedir = np.argwhere(pd.Series(ll).str.startswith("set\thome\t\t=").values)[0][0]
idx_start = ll[line_homedir].find('"')
idx_end = ll[line_homedir].find('"', idx_start+1)
dir_iraf = ll[line_homedir][idx_start+1:idx_end]    # Home directory recoded in the 'login.cl' file
print(dir_iraf)    # For check

# Importing IRAF
os.chdir(dir_iraf)
from pyraf import iraf
os.chdir(current_dir)

/data/jlee/DATA/TA/2022A/


# 1. Displaying the Images

In [4]:
dir_img = "Combined_images/"
imglist = [dir_img+"M105-g.fits", dir_img+"M105-i.fits",
           dir_img+"M100-g.fits", dir_img+"M100-i.fits"]
n_img = len(imglist)
imglist

['Combined_images/M105-g.fits',
 'Combined_images/M105-i.fits',
 'Combined_images/M100-g.fits',
 'Combined_images/M100-i.fits']

In [5]:
# You can also run this command in terminal.
ds9_options = "-scalemode zscale -scale lock yes -frame lock image "
names = ""
for i in np.arange(n_img):
    names += imglist[i]+" "
ds9_command = "ds9 "+ds9_options+names+"&"
print('Running "'+ds9_command+'" in the terminal...')
os.system(ds9_command)

Running "ds9 -scalemode zscale -scale lock yes -frame lock image Combined_images/M105-g.fits Combined_images/M105-i.fits Combined_images/M100-g.fits Combined_images/M100-i.fits &" in the terminal...


0

# 2. Running IRAF/Ellipse Task 

In [6]:
# IRAF/ellipse task is in the STSDAS package (IRAF external package)
iraf.stsdas()
iraf.stsdas.analysis()
iraf.stsdas.analysis.isophote()



      +------------------------------------------------------------+
      |       Space Telescope Science Data Analysis System         |
      |                   STSDAS Version 3.18.3                    |
      |                                                            |
      |   Space Telescope Science Institute, Baltimore, Maryland   |
      |   Copyright (C) 2014 Association of Universities for       |
      |            Research in Astronomy, Inc.(AURA)               |
      |       See stsdas$copyright.stsdas for terms of use.        |
      |         For help, send e-mail to help@stsci.edu            |
      |                                                            |
      +------------------------------------------------------------+


In [7]:
# For viewing help file of "ellipse" task
### IRAF.net website has recently stop its service... :(
### But you can still see the detailed description of input parameters of any tasks (but not so convenient) 
iraf.epar("ellipse")
# You have to see the detailed definition of each parameter in geompar, controlpar, samplepar, and magpar.
# iraf.epar("[TASK_NAME]")
### Click "[TASK_NAME] help" for viewing the parameter editor help browser

In [8]:
def fit_ellipse(input_image, output_table=None, interactive=False,
                model_image=None, residual_image=None, display=False, data_file=None,
                x0=100.0, y0=100.0, ellip0=0.1, pa0=45.0, sma0=10.0,  # geompar
                minsma=0.0, maxsma=50.0, step=1.0, linear=False, recenter=False,  # geompar
                minit=10, maxit=100, hcenter=False, hellip=False, hpa=False,  # controlpar
                usclip=3.0, lsclip=3.0, nclip=0,  # samplepar
                mag0=25.0,  # magpar
                backgr=0.0, interp='linear'):  # bmodel

    '''
    # --- basic input parameters --- #
    input_image - input image name ('[FILENAME].fits')
    output_table - output table name (default: '[FILENAME].tab')
    interactive - interactive (boolean, default: False)
    model_image - output model image (default: '[FILENAME]_mod.fits')
    residual_image - output residual image (default: '[FILENAME]_res.fits')
    data_file - output data file (default: '[FILENAME].dat')
    display - display the results or not? (boolean, default: False)
    
    # --- geompar set --- #
    x0, y0 - initial isophote center X, Y [pixel]
    ellip0, pa0 - initial ellipticity, position angle [degree]
    sma0 - initial semi-major axis (SMA) length [pixel]
    minsma - minimum SMA length for fitting [pixel] (default: 0.0)
    maxsma - maximum SMA length for fitting [pixel]
    step - SMA step between successive ellipses [pixel]
    linear - linear SMA step for fitting? (boolean, default: False)
    recenter - do you allow to re-center x0 & y0? (boolean, default: False)
    
    # --- controlpar set --- #
    minit - minimum iteration number at each step of SMA (default: 10)
    maxit - maximum iteration number at each step of SMA (default: 100)
    hcenter - do you want to hold center fixed? (boolean, default: False)
    hellip - do you want to hold ellipticity fixed? (boolean, default: False)
    hpa - do you want to hold position angle fixed? (boolean, default: False)
    
    # --- samplepar set --- #
    usclip - upper sigma-clip criterion (default: 3)
    lsclip - lower sigma-clip criterion (default: 3)
    nclip - iteration number for the sigma clipping (default: 0)
    
    # --- magpar set --- #
    mag0 - magnitude zeropoint (default: 25.0)
    
    # --- bmodel parameter set --- #
    backgr - background level for making model image (default: 0.0)
    interp - interpolation algorithm for model image ('nearest' OR 'linear' OR 'poly3' OR 'spline', default: 'linear')    
    '''
    
    iname = input_image.split('.fits')[0].split('/')[-1]    # Image name
    if (output_table is None):
        output_table = iname+'.tab'    # Output table name
    if (model_image is None):
        model_image = iname+'_mod.fits'    # Output model image name
    if (residual_image is None):
        residual_image = iname+'_res.fits'    # Output residual image name
    if (data_file is None):
        data_file = iname+'.dat'    # Output data file name 

    # Running IRAF/ellipse task
    os.system("rm -rfv "+output_table+" colnames.lis "+data_file)  # Reset by removing the output data
    kwargs = {"x0":x0, "y0":y0, "ellip0":ellip0, "pa0":pa0, "sma0":sma0,
              "minsma":minsma, "maxsma":maxsma, "step":step, "linear":"no", "recenter":"no",
              "minit":minit, "maxit":maxit, "hcenter":"no", "hellip":"no", "hpa":"no",
              "integrmode":"bi-linear", "usclip":usclip, "lsclip":lsclip, "nclip":nclip,
              "mag0":mag0, "refer":1.0E-5, "zerolevel":0.0}
    iraf.ellipse(input=input_image, output=output_table, interactive=interactive,
                 **kwargs)
    
    # Making model, residual images
    os.system("rm -rfv "+model_image+" "+residual_image)  # Reset by removing the output images
    iraf.bmodel(table=output_table, output=model_image, parent=input_image,
                backgr=backgr, interp=interp)    # bmodel task for model image
    
    iraf.imarith(input_image, "-", model_image, residual_image)    # input - model = residual
    
    if display:    # if display == True, DS9 will display input, model, and residual images.
        opt = " -scalemode zscale -scale lock yes -frame lock image "
        opt += " -tile grid mode manual -tile grid layout 3 1 "
        os.system("ds9 "+opt+input_image+" "+model_image+" "+residual_image+"&")
    
    # Reading output results
    ### output_table is not directly readable because it is a binary-format file :( 
    iraf.tlcol(output_table, nlist=1, Stdout='colnames.lis')    # Extracting column names
    iraf.tdump(table=output_table, columns="@colnames.lis", datafile=data_file)    # Making ASCII data file
    
    f = open("colnames.lis", "r")
    cc = f.readlines()
    f.close()

    colnames = []
    for line in cc:
        if not (line[0] == '#'):
            colnames.append(line.split(' ')[0])
#     print(colnames)    # column names array
    
    iso_tbl = np.genfromtxt(data_file, encoding="ascii", names=colnames)    # Reading data
    iso_df = pd.DataFrame(iso_tbl)
#     print(iso_df)    # output result data frame
    
    return iso_df

In [9]:
# For g-band image of M105
imgname = "Combined_images/M105-g.fits"
x_center, y_center = 455.0, 455.0    # depending on your object
r0 = 420.0    # maximum SMA

# --- Background estimation for determining backgroun level --- #
### (This is up to you! You do not have to do this if the background level in your images can be obviously determined.)
img = fits.getdata(imgname, ext=0)

x1d = np.arange(0, img.shape[1], 1)
y1d = np.arange(0, img.shape[0], 1)
xx, yy = np.meshgrid(x1d, y1d, sparse=True)
z = ((xx-x_center)**2.0 + (yy-y_center)**2.0 - r0**2.0)
sky_region = (z > 0.0)

avg, med, std = sigma_clipped_stats(img[sky_region], sigma=3.0)
sky_val, sky_sig = 3.0*med - 2.0*avg, std
print(sky_val, sky_sig)
# ---------- #

kwargs = {"x0":455.0, "y0":455.0, "ellip0":0.05, "sma0":5.0,
          "minsma":0.05, "maxsma":470, "step":0.05,
          "hcenter":False, "hellip":False, "hpa":False,
          "nclip":2, "mag0":22.5,
          "backgr":sky_val, "interp":"linear"}  # Here you can change input (default) parameter if needed!


iso_df = fit_ellipse(imgname, display=True, **kwargs)


# def fit_ellipse(input_image, output_table=None, interactive=False,
#                 model_image=None, residual_image=None, display=False, data_file=None,
#                 x0=100.0, y0=100.0, ellip0=0.1, pa0=45.0, sma0=10.0,  # geompar
#                 minsma=0.0, maxsma=50.0, step=1.0, linear=False, recenter=False,  # geompar
#                 minit=10, maxit=100, hcenter=False, hellip=False, hpa=False,  # controlpar
#                 usclip=3.0, lsclip=3.0, nclip=0,  # samplepar
#                 mag0=25.0,  # magpar
#                 backgr=0.0, interp='nearest'

(0.02016184872497758, 0.015442665365112712)
Running object locator... Done.
#
# Semi-    Isophote      Ellipticity     Position   Grad.  Data Flag Iter. Stop
# major      mean                         Angle      rel.                  code
# axis     intensity                                error
#(pixel)                                 (degree)
#
   5.00    33.55(  0.14) 0.084(0.003)  70.25( 1.16) 0.018   30   0    20    0
   5.25    32.44(  0.14) 0.087(0.003)  71.93( 1.06) 0.015   32   0    10    0
   5.51    31.26(  0.13) 0.087(0.003)  72.97( 0.97) 0.014   33   0    10    0
   5.79    30.11(  0.12) 0.088(0.002)  73.48( 0.83) 0.012   35   0    10    0
   6.08    28.95(  0.12) 0.089(0.002)  73.43( 0.81) 0.011   37   0    10    0
   6.38    27.83(  0.11) 0.090(0.002)  73.68( 0.71) 0.010   39   0    10    0
   6.70    26.70(  0.09) 0.093(0.002)  73.93( 0.57) 0.010   40   0    10    0
   7.04    25.53(  0.10) 0.091(0.002)  74.14( 0.62) 0.010   42   0    10    0
   7.39    24.42(  0.12) 0.0

   3.38    42.40(  0.23) 0.101(0.007)  68.47( 2.19) 0.032   21   0    10    0
   3.22    43.31(  0.26) 0.098(0.008)  66.60( 2.71) 0.043   20   0    10    0
   3.07    44.33(  0.24) 0.107(0.008)  68.21( 2.49) 0.045   19   0    10    0
   2.92    45.09(  0.27) 0.095(0.010)  66.66( 3.34) 0.049   18   0    10    0
   2.78    46.05(  0.28) 0.103(0.011)  67.61( 3.40) 0.055   17   0    10    0
   2.65    46.80(  0.25) 0.096(0.011)  65.38( 3.55) 0.061   16   0    10    0
   2.53    47.56(  0.29) 0.097(0.013)  64.34( 4.31) 0.065   16   0    10    0
   2.41    48.36(  0.31) 0.105(0.016)  70.59( 4.89) 0.085   15   0    10    0
   2.29    49.04(  0.33) 0.103(0.020)  68.83( 5.94) 0.101   14   0    10    0
   2.18    49.76(  0.35) 0.107(0.022)  68.89( 6.20) 0.100   13   0    10    0
   2.08    50.28(  0.35) 0.090(0.023)  68.89( 7.69) 0.107   13   0   100    2
   1.98    50.81(  0.35) 0.090(0.025)  68.89( 8.70) 0.122   13   0   100    2
   1.88    51.29(  0.32) 0.090(0.028)  68.89( 9.45) 0.135   13  